In [23]:
import pandas as pd

import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.utils import *

from custom_score.utils import cleanString
from sklearn.cluster import SpectralClustering, Birch
from hdbscan import HDBSCAN

import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

In [3]:
subset = dataset.iloc[:2, :]

### Corpus to Sentences

In [19]:
import spacy

In [20]:
def corpusToSentences(corpus):
    corpus_sentences = []
    nlp = spacy.load("en_core_web_sm")
    for indiv in corpus:
        doc = nlp(indiv)
        corpus_sentences.append([sent.text for sent in doc.sents])   
    return corpus_sentences

In [21]:
res = corpusToSentences(subset["text"].tolist())

In [22]:
print(res)

[["anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ].", 'although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ].', 'yet ,.', 'our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression.', 'overall , neuropsychiatric symptoms in pd have been shown to be negatively associated with cognitive performance.', 'for example , higher depression scores have been correlated with lower scores on the mini mental state exam ( mmse )', '[ 8 , 9 ] as well as tests of memory and executive functions ( e.g. , attention ) [ 1014 ].', 'likewise , apathy and anhedonia in pd patients have been associated with executive

### Token to sentences

In [25]:
def cleanMarkers(embs, labels, mode="all", ignore=["."]):
    """
    Removes vectors associated with noisy words such as stop words, punctuation, and BERT separator tokens.

    :param1 embs (list): List of words embeddings.
    :param2 labels (list): List of text token associated with each embedding.

    :output1 new_embs (list): Cleansed list of words embeddings.
    :output2 new_labels (list): Cleansed list of tokens.
    :output3 -1 (int): Error output.
    """
    token_indexes = [i for i in range(len(labels)) if (labels[i] != "[PAD]" and labels[i] != "[CLS]" and labels[i] != "[SEP]") or labels[i] in ignore]
    new_embs = [embs[i] for i in range(len(embs)) if i in token_indexes]
    new_labels = [labels[i] for i in range(len(labels)) if i in token_indexes]

    if mode == "all":
        return new_embs, new_labels
    elif mode == "emb":
        return new_embs
    elif mode == "lab":
        return new_labels
    else:
        return -1

In [27]:
o, l = tokenizeCorpus(subset["text"][0])
v = vectorizeCorpus(o)

Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors


In [28]:
v, l = cleanMarkers(v, l)

In [30]:
print(" ".join(l))

anxiety affects quality of life in those living with parkinson ' s disease ( pd ) more so than overall cognitive status , motor deficit ##s , ap ##athy , and depression [ 13 ] . although anxiety and depression are often related and coe ##xi ##st in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet , . our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains me ##ager and la ##gs far behind that of depression . overall , ne ##uro ##psy ##chia ##tric symptoms in pd have been shown to be negatively associated with cognitive performance . for example , higher depression scores have been correlated with lower scores on the mini mental state exam ( mm ##se ) [ 8 , 9 ] as well as tests of memory and executive functions ( e . g . , attention ) [ 101 ##4 ] . likewise , ap ##athy and an ##hed ##onia in pd patients have

In [32]:
sentenced_tokens = corpusToSentences([" ".join(l)])
print(sentenced_tokens)

[["anxiety affects quality of life in those living with parkinson ' s disease ( pd ) more so than overall cognitive status , motor deficit ##s , ap ##athy , and depression [ 13 ] .", 'although anxiety and depression are often related and coe ##xi ##st in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] .', 'yet , .', 'our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains me ##ager and la ##gs far behind that of depression .', 'overall , ne ##uro ##psy ##chia ##tric symptoms in pd have been shown to be negatively associated with cognitive performance .', 'for example , higher depression scores have been correlated with lower scores on the mini mental state exam ( mm ##se )', '[ 8 , 9 ] as well as tests of memory and executive functions ( e .', 'g .', ', attention )', '[ 101 ##4 ] .', 'likewise , ap ##athy and an

### Sentence Masks

In [36]:
mask = [i for i, sent in enumerate(sentenced_tokens[0]) for _ in sent.split(" ")]

In [37]:
mask

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,